<a href="https://colab.research.google.com/github/umabhargavi20/Robustness-Evaluation-of-CNN-ResNet-and-CLIP-Models-Against-Adversarial-Attacks/blob/main/CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/TML/ATTACKS_CIFAR_MNIST')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models, regularizers

In [ ]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [ ]:
print(f"X_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


In [ ]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

def plot_sample(X, y, index):
    plt.figure(figsize=(15, 2))
    plt.imshow(X[index])
    plt.xlabel(class_names[y[index].item()])
    plt.show()

In [ ]:
X_train = x_train.astype('float32') / 255.
X_test = x_test.astype('float32') / 255.

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
model = models.Sequential([

    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.3),

    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.4),

    layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.5),

    layers.GlobalAveragePooling2D(),

    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),

    layers.Dense(num_classes, activation='softmax')
])

NameError: name 'models' is not defined

In [ ]:
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

In [ ]:
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
               epochs=50,
               validation_data=(X_test, y_test),
               callbacks=[early_stopping])

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 53s 26ms/step - accuracy: 0.3534 - loss: 2.7977 - val_accuracy: 0.5477 - val_loss: 1.4450
Epoch 2/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 20ms/step - accuracy: 0.5740 - loss: 1.2699 - val_accuracy: 0.6171 - val_loss: 1.1812
Epoch 3/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 20ms/step - accuracy: 0.6445 - loss: 1.1061 - val_accuracy: 0.6009 - val_loss: 1.2859
Epoch 4/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 20ms/step - accuracy: 0.6959 - loss: 0.9823 - val_accuracy: 0.6794 - val_loss: 1.0486
Epoch 5/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.7250 - loss: 0.9058 - val_accuracy: 0.7300 - val_loss: 0.9643
Epoch 6/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.7476 - loss: 0.8497 - val_accuracy: 0.7294 - val_loss: 0.9346
Epoch 7/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 20ms/step - accuracy: 0.7582 - loss: 0.8132 - val_accuracy: 0.7587 - val_loss: 0.8327
Epoch 8/50
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.7742 - loss: 0.77

In [ ]:
model.evaluate(X_test,y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8727 - loss: 0.5033


[0.502621591091156, 0.8729000091552734]

In [ ]:
model.save("./model/CIFAR_model.keras")

In [ ]:
loaded_model = load_model('./model/CIFAR_model.keras')

In [ ]:
y_test.ndim

1

In [ ]:
# Check if y_test is one-hot encoded
if y_test.ndim > 1:  # If y_test has more than one dimension, it is one-hot encoded
    y_test_int = np.argmax(y_test, axis=1)  # Convert from one-hot to integer labels
else:
    y_test_int = y_test  # Already in integer format

# Apply FGSM to 10% of the test set
num_images = X_test.shape[0]
num_attack_images = int(num_images * 0.1)  # 10% of the test set
attack_indices = np.random.choice(num_images, num_attack_images, replace=False)
X_test_adv = np.copy(X_test)

In [ ]:
# FGSM Attack Function
def fgsm_attack(model, x, y, epsilon=0.1):
    x_var = tf.Variable(x)
    with tf.GradientTape() as tape:
        tape.watch(x_var)
        predictions = model(x_var)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y, predictions)  # use integer labels (y)
    gradient = tape.gradient(loss, x_var)
    perturbation = epsilon * tf.sign(gradient)
    x_adv = x + perturbation
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure the values are between 0 and 1
    return x_adv

In [ ]:
for i in attack_indices:
    X_test_adv[i] = fgsm_attack(loaded_model, X_test[i:i+1], y_test_int[i:i+1], epsilon=0.1)

In [ ]:
# Step 7: Evaluate on adversarial examples
loss_adv, accuracy_adv = loaded_model.evaluate(X_test_adv, y_test)
print(f"Test Loss (Adversarial): {loss_adv}")
print(f"Test Accuracy (Adversarial): {accuracy_adv}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7940 - loss: 1.1563
Test Loss (Adversarial): 1.1547452211380005
Test Accuracy (Adversarial): 0.79339998960495


In [ ]:
import random
from scipy.ndimage import shift
# Spatial Attack Function (Random Pixel Shifts)
def spatial_attack(x, shift_range=5):
    """
    Apply a spatial attack by shifting pixels in random directions.

    Args:
        x: Input image (1 sample).
        shift_range: Range for random pixel shifts (how many pixels to shift).

    Returns:
        Adversarial example.
    """
    # Randomly shift the image in both x and y directions
    shift_x = random.randint(-shift_range, shift_range)
    shift_y = random.randint(-shift_range, shift_range)

    # Apply the shift to the image
    x_adv = shift(x, (shift_x, shift_y, 0), mode='nearest')

    # Ensure the image is within [0, 1]
    x_adv = np.clip(x_adv, 0, 1)

    return x_adv

In [ ]:
X_test_adv = np.copy(X_test)
for i in attack_indices:
    X_test_adv[i] = spatial_attack(X_test[i], shift_range=5)

In [ ]:
# Evaluate the model on the mixed test set
loss, accuracy = loaded_model.evaluate(X_test_adv, y_test_int)
print(f"Test Loss (with 10% poisoned by Spatial Attack): {loss}")
print(f"Test Accuracy (with 10% poisoned by Spatial Attack): {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8712 - loss: 0.5070
Test Loss (with 10% poisoned by Spatial Attack): 0.5031977295875549
Test Accuracy (with 10% poisoned by Spatial Attack): 0.8723000288009644


In [ ]:
# PGD Attack Function
def pgd_attack(model, x, y, epsilon=0.1, alpha=0.01, num_iter=40):
    """
    Perform PGD attack on input data.

    Args:
        model: The target model.
        x: Input data (images).
        y: True labels.
        epsilon: Maximum perturbation.
        alpha: Step size for each iteration.
        num_iter: Number of iterations.

    Returns:
        Adversarial examples.
    """
    # Start with the original image
    x_adv = tf.Variable(x)

    for _ in range(num_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            predictions = model(x_adv)
            loss = tf.keras.losses.sparse_categorical_crossentropy(y, predictions)

        # Get the gradient of the loss with respect to the input image
        gradient = tape.gradient(loss, x_adv)

        # Apply the perturbation (sign of the gradient * step size)
        perturbation = alpha * tf.sign(gradient)

        # Update the adversarial image by adding the perturbation
        x_adv = x_adv + perturbation

        # Project the adversarial image back into the valid range [0, 1]
        x_adv = tf.clip_by_value(x_adv, 0, 1)

        # Ensure the perturbation is within the epsilon bound
        perturbation = tf.clip_by_value(x_adv - x, -epsilon, epsilon)
        x_adv = x + perturbation

        # Ensure the values are within [0, 1]
        x_adv = tf.clip_by_value(x_adv, 0, 1)

    return x_adv

In [ ]:
for i in attack_indices:
    X_test_adv[i] = pgd_attack(loaded_model, X_test[i:i+1], y_test_int[i:i+1], epsilon=0.1, alpha=0.01, num_iter=40)

In [ ]:
loss, accuracy = loaded_model.evaluate(X_test_adv, y_test_int)
print(f"Test Loss (with 10% poisoned by PGD): {loss}")
print(f"Test Accuracy (with 10% poisoned by PGD): {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7846 - loss: 7.8118
Test Loss (with 10% poisoned by PGD): 7.882907867431641
Test Accuracy (with 10% poisoned by PGD): 0.7835999727249146
